# CIE4140_Lecture_13_Part_2_Python #

## Let us calculate the natural and resonanse frequencies of a string with arbitrary (though linear) boundary conditions ##

The universal recipe is to substitute $w(x,t)=W(x) e^{i \omega t})

Strictly speaking, the natural frequencies are defined for undamped systems only. Therefore, we assume that there is no damping along the string nor in the boundaries.

In [208]:
import sympy as sp

The equation of motion in this case reads

In [209]:
w = sp.symbols('w',cls=sp.Function,real=True)
x,t = sp.symbols('x,t',real=True)
c = sp.symbols('c',real=True)
EQM = sp.diff(w(x,t),t,2)-c**2*sp.diff(w(x,t),x,2)
display(EQM)

-c**2*Derivative(w(x, t), (x, 2)) + Derivative(w(x, t), (t, 2))

Substituting $w(x,t)=W(x) e^{i \omega t}$, we obtain the following ordinary differential equation, in which $\omega$ is one of the natural frequencies


In [210]:
W = sp.symbols('W',cls=sp.Function,real=True)
omega = sp.symbols('omega',real=True)
EQM_Fr = sp.simplify(EQM.subs(w(x,t),W(x)*sp.exp(sp.I*omega*t)))
display(EQM_Fr)

-c**2*exp(I*omega*t)*Derivative(W(x), (x, 2)) - omega**2*W(x)*exp(I*omega*t)

The general solution to this equation reads

In [211]:
C1, C2 = sp.symbols('C1, C2',real=True)
W_sol = sp.dsolve(EQM_Fr,W(x)).rhs
display(W_sol)
W_sol = C1*sp.sin(omega*x/c)+C2*sp.cos(omega*x/c)
display(W_sol)

C1*sin(x*Abs(omega/c)) + C2*cos(omega*x/c)

C1*sin(omega*x/c) + C2*cos(omega*x/c)

Now we will consider different boundary conditions

1: Fixed-fixed string: $W(0)=0$, $W(L)=L$

In [212]:
L = sp.symbols('L',real=True)
BC_left =  W_sol.subs(x,0)
BC_right = W_sol.subs(x,L)
display(BC_left)
display(BC_right)

C2

C1*sin(L*omega/c) + C2*cos(L*omega/c)

When $\omega$ equals one of the natural frequencies, the determinant of the coefficient matrix composed of the two boundary conditions (i.e., the frequency equation) should be zero. 

Let us compose the coefficient matrix

In [213]:
Coeff_Matrix= sp.Matrix([[BC_left.diff(C1),BC_left.diff(C2)],
                         [BC_right.diff(C1),BC_right.diff(C2)]])
display(Coeff_Matrix)

Matrix([
[             0,              1],
[sin(L*omega/c), cos(L*omega/c)]])

In [214]:
Frequency_Equation = sp.det(Coeff_Matrix)
display(Frequency_Equation)

-sin(L*omega/c)

Now we search for the roots of the frequency equation, which, by definition, are the natural frequencies (in the undamped case these frequencies coincide with the resonance frequencies) 

Often, especially in the case of more complex boundary conditions, we introduce $\Omega_{dl}= {\Omega \over L} C$. For this dimensionless parameter, the roots of the frequency equation can be always found numerically 

In [216]:
Omega_dl = sp.symbols('Omega_dl',real=True)
Frequency_Equation_DL = Frequency_Equation.subs(omega,Omega_dl/L*c)
display(Frequency_Equation_DL)

-sin(Omega_dl)

Obviously, independently of the wave speed and the string length $\Omega_{dl}=\pi n$, where $n$ is an integer


In [217]:
sol = sp.solveset(sp.Eq(Frequency_Equation_DL,0),Omega_dl)
display(sol)

Union(ImageSet(Lambda(_n, 2*_n*pi + pi), Integers), ImageSet(Lambda(_n, 2*_n*pi), Integers))

Leta us find values of $\beta$ also numerically

In [218]:
Dimensionless_Natural_Frequency = []
step = 0.01
for i in range(5000):
    xi = i * step
    left_value = Frequency_Equation_DL.subs(Omega_dl,xi)
    right_value = Frequency_Equation_DL.subs(Omega_dl,xi+step)
    if left_value*right_value < 0:
        Dimensionless_Natural_Frequency.append(xi+step/2)
        
display(Dimensionless_Natural_Frequency)

[3.145,
 6.285,
 9.425,
 12.565000000000001,
 15.705000000000002,
 18.845,
 21.995,
 25.134999999999998,
 28.275,
 31.415,
 34.55500000000001,
 37.695,
 40.845000000000006,
 43.98500000000001,
 47.125]

2: Fixed-free string: $W(0)=0, {\left. {{{dW\left( x \right)} \over {dx}}} \right|_{x = L}} = 0$

In [219]:
BC_left =  W_sol.subs(x,0)
BC_right = W_sol.diff(x).subs(x,L)
display(BC_left)
display(BC_right)

C2

C1*omega*cos(L*omega/c)/c - C2*omega*sin(L*omega/c)/c

In [220]:
Coeff_Matrix= sp.Matrix([[BC_left.diff(C1),BC_left.diff(C2)],
                         [BC_right.diff(C1),BC_right.diff(C2)]])
display(Coeff_Matrix)

Matrix([
[                     0,                       1],
[omega*cos(L*omega/c)/c, -omega*sin(L*omega/c)/c]])

In [221]:
Frequency_Equation = sp.det(Coeff_Matrix)
display(Frequency_Equation)

-omega*cos(L*omega/c)/c

In [222]:
Frequency_Equation_DL = Frequency_Equation.subs(omega,Omega_dl/L*c)
display(Frequency_Equation_DL)

-Omega_dl*cos(Omega_dl)/L

In [223]:
sol = sp.solveset(sp.Eq(Frequency_Equation_DL,0),Omega_dl)
display(sol)

Union({0}, ImageSet(Lambda(_n, 2*_n*pi + pi/2), Integers), ImageSet(Lambda(_n, 2*_n*pi + 3*pi/2), Integers))

3: Fixed-{Elastically Supported Inertial End}: 
- ${w\left( {0,t} \right) = 0}$
- ${\left. {T{{\delta w\left( {x,t} \right)} \over {\delta x}}} \right|_{x = L}} =  - m{\left. {{{{\delta ^2}w\left( {x,t} \right)} \over {\delta {x^2}}}} \right|_{x = L}} + kw\left( {x,t} \right)$

In [257]:
T, m, k = sp.symbols('T, m, k',real=True)
BC_left =  W_sol.subs(x,0)
BC_right = T*sp.diff(w(x,t),x)+m*sp.diff(w(x,t),t,2)+k*w(x,t)
display(BC_left)
display(BC_right)

C2

T*Derivative(w(x, t), x) + k*w(x, t) + m*Derivative(w(x, t), (t, 2))

In [258]:
BC_right = sp.simplify(BC_right.subs(w(x,t),W_sol*sp.exp(sp.I*omega*t))).subs(x,L)
display(BC_right)

T*(C1*omega*cos(L*omega/c)/c - C2*omega*sin(L*omega/c)/c)*exp(I*omega*t) + k*(C1*sin(L*omega/c) + C2*cos(L*omega/c))*exp(I*omega*t) - m*omega**2*(C1*sin(L*omega/c) + C2*cos(L*omega/c))*exp(I*omega*t)

$e^{i \Omega t}$ is not dropped, so can dropped manually, not necessary:

In [259]:
BC_right = BC_right.subs(sp.exp(sp.I*omega*t),1)
display(BC_right)

T*(C1*omega*cos(L*omega/c)/c - C2*omega*sin(L*omega/c)/c) + k*(C1*sin(L*omega/c) + C2*cos(L*omega/c)) - m*omega**2*(C1*sin(L*omega/c) + C2*cos(L*omega/c))

In [260]:
Coeff_Matrix= sp.Matrix([[BC_left.diff(C1),BC_left.diff(C2)],
                         [BC_right.diff(C1),BC_right.diff(C2)]])
display(Coeff_Matrix)

Matrix([
[                                                                      0,                                                                        1],
[T*omega*cos(L*omega/c)/c + k*sin(L*omega/c) - m*omega**2*sin(L*omega/c), -T*omega*sin(L*omega/c)/c + k*cos(L*omega/c) - m*omega**2*cos(L*omega/c)]])

In [261]:
Frequency_Equation = sp.det(Coeff_Matrix)
display(Frequency_Equation)

(-T*omega*cos(L*omega/c) - c*k*sin(L*omega/c) + c*m*omega**2*sin(L*omega/c))/c

This equation should be solved numerically. The natural frequencies depend both on the string parameters and the mass and stiffness of the boundary

In [269]:
Frequency_Equation_subs = Frequency_Equation.subs([(L,1),(c,2),(T,10),(k,20),(m,7)])

In [270]:
sol = sp.solveset(sp.Eq(Frequency_Equation_subs,0),omega)
display(sol)

ConditionSet(omega, Eq(7*omega**2*sin(omega/2) - 5*omega*cos(omega/2) - 20*sin(omega/2), 0), Complexes)

In [271]:
Dimensionless_Natural_Frequency = []
step = 0.01
for i in range(5000):
    xi = i * step
    left_value = Frequency_Equation_subs.subs(omega,xi)
    right_value = Frequency_Equation_subs.subs(omega,xi+step)
    if left_value*right_value < 0:
        Dimensionless_Natural_Frequency.append(xi+step/2)
        
display(Dimensionless_Natural_Frequency)

[1.9449999999999998,
 6.515,
 12.685,
 18.925,
 25.185,
 31.465,
 37.73500000000001,
 44.015]